<a href="https://colab.research.google.com/github/Sachinsatya/data_insights-/blob/main/ImageDataGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Last Week we connected Kaggle to Colab and we ran our first competition, Dogs-Vs-Cats
# Somehow while loading the data into an array and while normalizing the Colab crashed & the reason was OOM 
# OOM - Out of Memory Error, it means that your complete data cant be loaded in Colabs RAM

# Possible Solutions - 
# 1. Decrease Data 
# 2. Increase RAM


# BOTH ARE NOT VIABLE

In [ ]:
# Solution is dont read data untill it is required! 

In [ ]:
# In our last sesssion we made the model - 
# Connect Kaggle to Colab 
# Download Data & UNzip  (DATASET of 25000 Images)
# READ IMAGES AND LOAD THEM IN AN ARRAY (loading it in RAM) 
# Pre Process the Data 
# ---------OOM ERROR --------------
# Build the model (build the skeleton)
# Compile it 
# Run it (Batch Size of 32)
#   Epoch 1 - 
#     Batch 1 -  
#         Feed Forward  - ( This is technically the very first time when data was required)
#         Error Calc    - 
#         Back Prop     -
#     Batch 2 - 
#     Batch 3 -  

In [ ]:
# We will not read all the images, rather we will read them on the fly when required for training
# and even do their preprocessing on the fly

# Connect Kaggle with Colab
# Download data (25K images)
# Build the model (build the skeleton)
# Compile 
# Run the model - (batch size of 32, 782 Batches)
# Epoch 1 - 
#   Batch 1 - 
#       Read 32 images and load them in an array 
#       Pre Process the data 
#       Feed Forward 
#       Error Calc
#       Back Propagation 
#       Delete these 32 images from RAM
#   Batch 2 - 
#       Read next 32 images and load them in an array 
#       Pre Process the data 
#       Feed Forward 
#       Error Calc 
#       Back Propagation 
#       Delete these 32 images from RAM 

In [ ]:
import numpy as np
def myFunc(x_array):
  return x_array + 1

In [ ]:
myFunc(np.array([1,2,3]))

array([2, 3, 4])

In [ ]:
# Generator 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2, os

In [ ]:
import kaggle

OSError: ignored

In [ ]:
! mv kaggle.json /root/.kaggle

In [ ]:
import kaggle

In [ ]:
! kaggle competitions download -c dogs-vs-cats

 99% 802M/812M [00:03<00:00, 233MB/s]
100% 812M/812M [00:03<00:00, 235MB/s]


In [ ]:
! unzip -q dogs-vs-cats.zip

In [ ]:
! unzip -q train.zip

In [ ]:
# Image Data Generator has a specific format for aliging images 
# Master_Folder
#   Category_1
#       Image_11
#       Image_12
#       Image_13
#       ...
#   Category_2
#     Image_21
#     Image_22
#     Image_23
#     ... 


# Master_data
#|-  Cat
#     cat_image_1.jpg
#     cat_image_2.jpg
#     ...
#|-  Dog
#     dog_image_1.jpg
#     dog_image_2.jpg

In [ ]:
# creating folder structure
os.mkdir("master_data")
os.mkdir("master_data/cat")
os.mkdir("master_data/dog")

In [ ]:
# copy these files from train folder and paste them in CAT or DOG folder accordingly
src_folder = "train/"
dest_cat = "master_data/cat/"
dest_dog = "master_data/dog/"

import shutil # python library which will help us in copying files

In [ ]:
# iterate over the train folder 
for filename in os.listdir(src_folder):
  if "dog" in filename:
    shutil.copy(src_folder + filename, dest_dog)
  else:
    shutil.copy(src_folder + filename, dest_cat)

In [ ]:
os.listdir("master_data")

['dog', 'cat']

In [ ]:
len(os.listdir("master_data/dog"))

12500

In [ ]:
len(os.listdir("master_data/cat"))

12500

In [ ]:
# master_data
# |- cat
#     |- ...images
# |- dog
#     |- ...images

In [ ]:
batch_size = 128

In [ ]:
# Scaling the images
idg = tf.keras.preprocessing.image.ImageDataGenerator(validation_split= 0.1, rescale=1/255.0)

In [ ]:
# Resizing the images to (150, 150)
train_idg = idg.flow_from_directory(directory = "master_data", target_size = (150, 150), batch_size=batch_size, subset="training")

Found 22500 images belonging to 2 classes.


In [ ]:
val_idg = idg.flow_from_directory(directory = "master_data", target_size=(150, 150), batch_size=batch_size, subset="validation")

Found 2500 images belonging to 2 classes.


In [ ]:
# del model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Input((150, 150, 3), name="Input"))
model.add(tf.keras.layers.Flatten(name="Flat"))
model.add(tf.keras.layers.Dense(100, activation=tf.keras.activations.relu, name="HiddenLayer"))
model.add(tf.keras.layers.Dense(2, activation=tf.keras.activations.softmax, name="Output"))

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Flat (Flatten)              (None, 67500)             0         
                                                                 
 HiddenLayer (Dense)         (None, 100)               6750100   
                                                                 
 Output (Dense)              (None, 2)                 202       
                                                                 
Total params: 6,750,302
Trainable params: 6,750,302
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer=tf.keras.optimizers.SGD(), loss=tf.keras.losses.categorical_crossentropy, metrics=["acc"])

In [ ]:
model.fit(train_idg, batch_size=batch_size, 
          epochs=10, 
          validation_data=val_idg)

Epoch 1/10
176/176 [==============================] - 126s 712ms/step - loss: 0.7366 - acc: 0.5432 - val_loss: 0.6747 - val_acc: 0.5712
Epoch 2/10
176/176 [==============================] - 121s 688ms/step - loss: 0.6681 - acc: 0.5885 - val_loss: 0.6699 - val_acc: 0.5864
Epoch 3/10
176/176 [==============================] - 115s 654ms/step - loss: 0.6572 - acc: 0.6092 - val_loss: 0.6764 - val_acc: 0.5808
Epoch 4/10
176/176 [==============================] - 124s 706ms/step - loss: 0.6517 - acc: 0.6164 - val_loss: 0.6624 - val_acc: 0.6024
Epoch 5/10
176/176 [==============================] - 125s 707ms/step - loss: 0.6453 - acc: 0.6261 - val_loss: 0.6617 - val_acc: 0.6008
Epoch 6/10
176/176 [==============================] - 128s 728ms/step - loss: 0.6415 - acc: 0.6289 - val_loss: 0.6745 - val_acc: 0.5856
Epoch 7/10
176/176 [==============================] - 113s 639ms/step - loss: 0.6407 - acc: 0.6320 - val_loss: 0.6517 - val_acc: 0.6140
Epoch 8/10
128/176 [====================>.......

KeyboardInterrupt: ignored

In [ ]:
# Total Images - 25000 
# Training Images - 22500
# Validation Images - 2500

# Batch Size - 32 
# 22500 / 32 = 703.125 = ~704

In [ ]:
model.save("dogVscat_model")